In [ ]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy


class GeometricShape:
    def __init__(self, shape_type, width, height):
        self.shape_type = shape_type
        self.width = width
        self.height = height
        self.color = (255, 255, 255)
        self.opacity = 1
        self.reset()

    def reset(self):
        if self.shape_type == 'rectangle':
            self.size = (np.random.randint(50, 100), np.random.randint(50, 100))
            self.position = self.get_rectangle_position()
            self.bounce_offset = 0
            self.thickness = np.random.randint(2, 5)  
        elif self.shape_type == 'line':
            self.thickness = np.random.randint(2, 5)
            self.position = np.random.randint(0, self.width)
            self.height_percentage = 0
            self.bounce_offset = 0
        self.bounce_speed = 0

    def get_rectangle_position(self):
        positions = [
            (0, self.height - self.size[1]),  # 左下
            (self.width - self.size[0], self.height - self.size[1]),  # 右下
            (0, 0),  # 左上
            (self.width - self.size[0], 0)  # 右上
        ]
        return positions[np.random.randint(0, 4)]

    def update(self, energy, current_onset, max_onset):
        if self.shape_type == 'rectangle':
            self.size = (int(self.size[0] * (1 + energy * 0.05)),
                         int(self.size[1] * (1 + energy * 0.05)))
            if self.size[0] > self.width // 3 or self.size[1] > self.height // 3:
                self.reset()
        
        elif self.shape_type == 'line':
            self.height_percentage = min(1, self.height_percentage + energy * 0.1)
        
        self.bounce_speed = current_onset / max_onset * 20
        self.bounce_offset = int(np.sin(self.bounce_speed) * 20)

    def draw(self, frame):
        if self.shape_type == 'rectangle':
            bounced_position = (self.position[0], self.position[1] + self.bounce_offset)
            cv2.rectangle(frame,
                          bounced_position,
                          (bounced_position[0] + self.size[0], bounced_position[1] + self.size[1]),
                          self.color, self.thickness)  # 線の太さを適用
        elif self.shape_type == 'line':
            start_point = (self.position, self.height)
            end_point = (self.position, int(self.height * (1 - self.height_percentage)))
            
            # 振動するバウンス効果を適用
            bounced_start = (start_point[0] + int(np.sin(self.bounce_speed) * 10), start_point[1])
            bounced_end = (end_point[0] + int(np.sin(self.bounce_speed) * 10), end_point[1])
            cv2.line(frame, bounced_start, bounced_end, self.color, self.thickness)
        return frame


def create_shapes(width, height):
    shapes = [GeometricShape('rectangle', width, height) for _ in range(4)]
    line_positions = np.linspace(width // 4, 3 * width // 4, 3, dtype=int)
    shapes.extend([GeometricShape('line', width, height) for _ in line_positions])
    # 各lineの位置を固定
    for i, shape in enumerate(shapes[4:]):
        shape.position = line_positions[i]
    return shapes

# 以下の関数は変更なし
def create_particles(num_particles, width, height):
    particles = np.random.rand(num_particles, 4)
    particles[:, 0] *= width
    particles[:, 1] *= height
    particles[:, 2] = np.random.randint(2, 5, num_particles)
    particles[:, 3] = np.random.randint(1, 5, num_particles)
    return particles

def create_frosted_glass_blocks(width, height, block_size_range=(50, 150), num_blocks=10):
    blocks = []
    for _ in range(num_blocks):
        block_width = np.random.randint(block_size_range[0], block_size_range[1])
        block_height = np.random.randint(block_size_range[0], block_size_range[1])
        block_x = np.random.randint(0, width - block_width)
        block_y = np.random.randint(0, height - block_height)
        blocks.append((block_x, block_y, block_width, block_height))
    return blocks

def update_particles(particles, width, height, energy):
    particles[:, 1] -= particles[:, 3] * energy
    reset = particles[:, 1] < 0
    particles[reset, 1] = height
    particles[reset, 0] = np.random.randint(0, width, np.sum(reset))
    return particles

def draw_particles(frame, particles, color):
    for x, y, size, _ in particles:
        cv2.circle(frame, (int(x), int(y)), int(size), color, -1)

def apply_blur_effect(frame, energy):
    blur_amount = max(1, int(energy * 2))
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

def apply_bounce_effect(frame, current_onset, max_onset, original_height):
    bounce_amount = int(30 * current_onset / max_onset)
    if bounce_amount > 0:
        frame = frame[bounce_amount:-bounce_amount, :]
    return cv2.resize(frame, (frame.shape[1], original_height)), bounce_amount

def apply_frosted_glass_effect(frame, blocks, blur_strength=15):
    result = frame.copy()
    for x, y, w, h in blocks:
        roi = result[y:y+h, x:x+w]
        blurred = cv2.GaussianBlur(roi, (blur_strength, blur_strength), 0)
        mean = cv2.mean(blurred)[0]
        contrast_enhanced = cv2.addWeighted(blurred, 1.5, blurred, 0, mean * -0.5)
        edge_enhanced = cv2.addWeighted(contrast_enhanced, 1.5, cv2.Laplacian(contrast_enhanced, cv2.CV_8U), 0.5, 0)
        alpha = 0.7
        result[y:y+h, x:x+w] = cv2.addWeighted(roi, alpha, edge_enhanced, 1-alpha, 0)
    return result

def apply_glitch_effect(frame, strength=10):
    height, width, _ = frame.shape
    glitch_frame = frame.copy()
    num_slices = np.random.randint(1, strength)
    for _ in range(num_slices):
        slice_height = np.random.randint(1, height // strength)
        start_y = np.random.randint(0, height - slice_height)
        start_x = np.random.randint(-strength, strength)
        end_x = width + start_x
        if start_x > 0:
            end_x = min(end_x, width)
            glitch_frame[start_y:start_y + slice_height, start_x:end_x] = frame[start_y:start_y + slice_height, :end_x - start_x]
        else:
            start_x = max(start_x, -width)
            glitch_frame[start_y:start_y + slice_height, :end_x] = frame[start_y:start_y + slice_height, -start_x:]
    return glitch_frame

def apply_trailing_effect(frame, previous_frames):
    alpha = 0.7
    result = frame.copy()
    for prev_frame in previous_frames:
        result = cv2.addWeighted(result, alpha, prev_frame, 1 - alpha, 0)
    return result

def apply_color_shift(frame, shift_amount):
    b, g, r = cv2.split(frame)
    b = np.roll(b, shift_amount, axis=1)
    r = np.roll(r, -shift_amount, axis=1)
    return cv2.merge([b, g, r])

def create_music_visualizer(image_path, audio_path, output_path):
    y, sr = librosa.load(audio_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    duration = librosa.get_duration(y=y, sr=sr)
    fps = 30
    n_frames = int(duration * fps)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64, fmax=8000, hop_length=sr // fps)
    S_dB = librosa.power_to_db(S, ref=np.max)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=sr // fps)

    height, width, _ = img.shape
    particles = create_particles(100, width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('temp_output.mp4', fourcc, fps, (width, height))

    prev_bars = np.zeros(64)
    zoom_factor = 1.0
    max_onset = np.max(onset_env)
    previous_frames = []

    shapes = create_shapes(width, height)
    frosted_blocks = create_frosted_glass_blocks(width, height, block_size_range=(50, 150), num_blocks=15)

    # イントロブラーの適用
    intro_duration = 3  # 秒
    intro_frames = int(intro_duration * fps)
    max_blur = 80

    for frame_num in range(n_frames):
        audio_idx = int(frame_num * len(y) / n_frames)
        chunk = y[audio_idx:audio_idx + sr // fps]

        spec_frame = S_dB[:, min(frame_num, S_dB.shape[1] - 1)]
        current_onset = onset_env[min(frame_num, len(onset_env) - 1)]
        energy = np.mean(np.abs(chunk)) * 10

        frame = img.copy()

        # イントロブラーエフェクトの適用
        if frame_num < intro_frames:
            blur_amount = max_blur - int((frame_num / intro_frames) * max_blur)
            frame = cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

        particles = update_particles(particles, width, height, energy)
        draw_particles(frame, particles, (255, 255, 255))

        bar_width = int(width / (64 + 1.5))  
        bar_spacing = int(bar_width / 2)
        max_bar_height = height // 6

        target_heights = np.interp(spec_frame, [S_dB.min(), S_dB.max()], [0, max_bar_height])
        prev_bars = prev_bars * 0.7 + target_heights * 0.3
        bar_heights = prev_bars.astype(int)

        overlay = frame.copy()

        for j, bar_height in enumerate(bar_heights):
            x1 = j * (bar_width + bar_spacing)
            x2 = x1 + bar_width

            cv2.rectangle(overlay,
                          (x1, height // 2 - bar_height),
                          (x2, height // 2),
                          (255, 255, 255, 100), 
                          -1)
            cv2.rectangle(overlay,
                          (x1, height // 2),
                          (x2, height // 2 + bar_height),
                          (255, 255, 255, 100),  
                          -1)
            
            cv2.rectangle(overlay,
                          (x1, height // 2 - bar_height),
                          (x2, height // 2 + bar_height),
                          (255, 255, 255, 255), 
                          1)  

        alpha = 0.7  
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        frame = apply_blur_effect(frame, energy)
        target_zoom = 1 + 0.05 * current_onset / max_onset
        zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3
        scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)
        start_y = (scaled_frame.shape[0] - height) // 2
        start_x = (scaled_frame.shape[1] - width) // 2
        frame = scaled_frame[start_y:start_y + height, start_x:start_x + width]
        frame, bounce_offset = apply_bounce_effect(frame, current_onset, max_onset, height)

        frame = apply_color_shift(frame, int(energy * 10))

        if np.random.rand() < 0.2:
            frame = apply_glitch_effect(frame)

        if len(previous_frames) > 5:
            previous_frames.pop(0)
        previous_frames.append(frame.copy())
        frame = apply_trailing_effect(frame, previous_frames)

        for shape in shapes:
            shape.update(energy, current_onset, max_onset)
            frame = shape.draw(frame)

        frame = apply_frosted_glass_effect(frame, frosted_blocks, blur_strength=85)

        video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    video.release()

    video = mpy.VideoFileClip('temp_output.mp4')
    audio = mpy.AudioFileClip(audio_path).set_duration(video.duration)
    final_video = video.set_audio(audio)
    final_video.write_videofile(output_path, codec="libx264", audio_codec="aac")

# 使用例
image_path = 'tmpnxrvzdcb.png'
audio_path = 'Moonlit Siren.mp3'
output_path = 'output_video.mp4'
create_music_visualizer(image_path, audio_path, output_path)